Есть два набора данных (во вложении): визиты пользователей на сервис и данные по покупкам. У каждого визита есть источник, а также устройство, с которого заходил пользователь.

purchases – покупки. 
visits – визиты. 
```
Обозначения в файлах:
start_dt — дата старта визита,
start_ts — дата и время старта визита,
end_ts — дата окончания визита,
uid — идентификатор пользователя,
source_id — id источника визита,
device — устройство,
buy_ts — дата и время покупки,
revenue — доход в y.e.
```
```
Необходимо определить:

1)	Какая когорта пользователей (по месяцу первой покупки) принесла наибольший средний чек на пользователя, какая наименьший.
2)	Какие аномалии в трафике видите, если они есть.
3)	Какой LTV на каждую когорту пользователей (по месяцу первой покупки)?
4)	С какого source_id мы получили больший доход помесячно, учитывая, что покупку относим к id источника последнего перед покупкой визита.
```

Просьба написать sql-запросы и визуализировать данные с помощью любого инструмента.


In [2]:
import pandas as pd

In [3]:
df1p = pd.read_excel('Источники переделанные в excel/purchases.xlsx')
df2v = pd.read_excel('Источники переделанные в excel/visits.xlsx')

In [25]:
#Просмотр типов данных: 
df1p.info()
df2v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75716 entries, 0 to 75715
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   buy_ts   75716 non-null  datetime64[ns]
 1   uid      75716 non-null  int64         
 2   revenue  75716 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720679 entries, 0 to 720678
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   device     720679 non-null  object        
 1   end_ts     720679 non-null  object        
 2   source_id  720679 non-null  int64         
 3   start_dt   720679 non-null  datetime64[ns]
 4   start_ts   720679 non-null  object        
 5   uid        720679 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 33.0+ MB


In [27]:
#Просмотр дат в визитах:
df2v.head()

,device,end_ts,source_id,start_dt,start_ts,uid
0,desktop,2016-11-30T11:04:17Z,2,2016-11-30,2016-11-30T10:53:49Z,3758486201
1,desktop,2016-11-10T23:16:57Z,4,2016-11-10,2016-11-10T23:08:04Z,2933941628
2,desktop,2016-10-19T18:51:56Z,1,2016-10-19,2016-10-19T18:48:16Z,1465872791
3,desktop,2017-01-12T20:51:55Z,9,2017-01-12,2017-01-12T20:50:10Z,2751875787
4,touch,2016-07-23T19:17:19Z,4,2016-07-23,2016-07-23T19:10:57Z,3492372810


In [29]:
#Форматирование к одному формату данных: 

df2v['start_ts'] = pd.to_datetime(df2v['start_ts'])
df2v['end_ts'] = pd.to_datetime(df2v['end_ts'])

In [37]:
#Просмотр типов данных: 
df1p.info()
df2v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75716 entries, 0 to 75715
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   buy_ts   75716 non-null  datetime64[ns]
 1   uid      75716 non-null  int64         
 2   revenue  75716 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720679 entries, 0 to 720678
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   device     720679 non-null  object             
 1   end_ts     720679 non-null  datetime64[ns, UTC]
 2   source_id  720679 non-null  int64              
 3   start_dt   720679 non-null  datetime64[ns]     
 4   start_ts   720679 non-null  datetime64[ns, UTC]
 5   uid        720679 non-null  int64              
dtypes: datetime64[ns, UTC](2), datetime64[ns](1), int64(2), obj

(720679, 6)

In [31]:
#Просмотр дат в визитах:
df2v.head()

,device,end_ts,source_id,start_dt,start_ts,uid
0,desktop,2016-11-30 11:04:17+00:00,2,2016-11-30,2016-11-30 10:53:49+00:00,3758486201
1,desktop,2016-11-10 23:16:57+00:00,4,2016-11-10,2016-11-10 23:08:04+00:00,2933941628
2,desktop,2016-10-19 18:51:56+00:00,1,2016-10-19,2016-10-19 18:48:16+00:00,1465872791
3,desktop,2017-01-12 20:51:55+00:00,9,2017-01-12,2017-01-12 20:50:10+00:00,2751875787
4,touch,2016-07-23 19:17:19+00:00,4,2016-07-23,2016-07-23 19:10:57+00:00,3492372810


In [53]:
#Данные до Join:
print('Покупки', df1p.shape)
print('Визиты', df2v.shape)

Покупки (75716, 3)
Визиты (720679, 6)


In [54]:
#Join файлов:
tabl1 = pd.merge(df1p, df2v, on='uid', how='left')

In [55]:
#Данные после Join:
print('table1', tabl1.shape)

table1 (5325086, 8)
